# Export files for GNPS

With pyOpenMS you can automatically generate all files needed for GNPS
Feature-Based Molecular Networking (FBMN) and Ion Identity Molecular
Networking (IIMN).

Pre-requisites are your input mzML files and a `ConsensusMap`, generated
by an [untargeted metabolomics pre-processing
workflow](metabolomics_preprocessing.ipynb). Ensure that MS2 data has
been mapped to the `FeatureMap` objects with `IDMapper`. For IIMN adduct
detection must have been performed on the `FeatureMap` objects during
pre-processing with `MetaboliteFeatureDeconvolution`.

First, download two example `mzML` files that have been map aligned
based on a feature map alignment.

In [ ]:
from urllib.request import urlretrieve
gh = "https://raw.githubusercontent.com/OpenMS/pyopenms-docs/master"
urlretrieve (gh + "/src/data/Metabolomics_1_aligned.mzML", "Metabolomics_1_aligned.mzML")
urlretrieve (gh + "/src/data/Metabolomics_2_aligned.mzML", "Metabolomics_2_aligned.mzML")
urlretrieve (gh + "/src/data/UntargetedMetabolomics.consensusXML", "UntargetedMetabolomics.consensusXML")

In [ ]:
from pyopenms import *

mzML_files = ["Metabolomics_1_aligned.mzML", "Metabolomics_2_aligned.mzML"]

consensusXML_file = "UntargetedMetabolomics.consensusXML"

Since GNPS only works with features that contain MS2 fragmentation
spectra, the first step is to filter out features from your
`ConsensusMap` that have no MS2 spectra annotated.

In [ ]:
consensus_map = ConsensusMap()
ConsensusXMLFile().load(consensusXML_file, consensus_map)
filtered_map = ConsensusMap(consensus_map)
filtered_map.clear(False)
for feature in consensus_map:
    if feature.getPeptideIdentifications():
        filtered_map.push_back(feature)

consensusXML_file = "filtered.consensusXML"
ConsensusXMLFile().store(consensusXML_file, filtered_map)

Now you can export your all files for FBMN and IIMN.

In [ ]:
# for FFBM
GNPSMGFFile().store(String(consensusXML_file), [file.encode() for file in mzML_files], String("MS2data.mgf")) 
GNPSQuantificationFile().store(consensus_map, "FeatureQuantificationTable.txt")
GNPSMetaValueFile().store(consensus_map, "MetaValueTable.tsv")

# for IIMN
IonIdentityMolecularNetworking().annotateConsensusMap(consensus_map)
IonIdentityMolecularNetworking().writeSupplementaryPairTable(consensus_map, "SupplementaryPairTable.csv")